# Fraud Detection Data Preprocessing Notebook

In [ ]:
dataset_copy = dataset.copy()

In [ ]:
df_fraud = pd.DataFrame(dataset_copy['train'])

In [ ]:
def data_preprocessing(data):
    def value_encode(data_field):
        return data_field.apply(lambda field: float(sum(bytearray(str(field).encode('utf-8')))))
    
    data_copy = data.copy()

    data_copy.drop('isFlaggedFraud', inplace=True, axis=1)
    data_copy.drop('step', inplace=True, axis=1)

    data_copy['type'] = value_encode(data_copy['type'])
    data_copy['nameOrig'] = value_encode(data_copy['nameOrig'])
    data_copy['nameDest'] = value_encode(data_copy['nameDest'])

    return data_copy